### Load libraries

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from itertools import islice
import torch
import os
import wandb
from gluonts.dataset.repository import get_dataset, dataset_names
from gluonts.dataset.util import to_pandas
from gluonts.dataset.common import ListDataset
from tqdm.autonotebook import tqdm
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.pandas import PandasDataset
from lag_llama.gluon.estimator import LagLlamaEstimator

/tmp/ipykernel_18695/1724617453.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### train_ds and test_ds in ListDatasets ✅ 

In [2]:
train_df = pd.read_csv(f'train_df_14.csv', index_col=0)
train_df_std = (train_df - train_df.mean())/train_df.std()
train_data = [{"start": train_df_std.index[0], "target": train_df_std[i].values} for i in train_df_std.columns]
train_ds = ListDataset(data_iter=train_data, freq='1s') # training dataset

In [3]:
test_df = pd.read_csv(f'test_df_14.csv', index_col=0)
test_df_std = (test_df - test_df.mean())/test_df.std()
test_data = [{"start": test_df_std.index[0], "target": test_df_std[i].values} for i in test_df_std.columns]
test_ds = ListDataset(data_iter=test_data, freq='1s') # testing dataset

### define sweeper in wandb

In [4]:
# Define the search space
sweep_config = {
    "method": "grid",
    "parameters": {
        "lr": {"values": [5e-4, 10e-4, 5e-5 ,10e-5]}, #4 bigger magnitude 10-2 ... 10-6, lr_scheduler
        "batch_size": {"values": [2, 8, 16, 32, 64, 96]}, #6
        "context_length": {"values": [32, 64, 96, 128]} #4
    }
}
sweep_id = wandb.sweep(sweep_config, project="lag_llama_04_07") # a new sweep is created in the sweep space under my w&b account

Create sweep with ID: 8bk4uije
Sweep URL: https://wandb.ai/junboma/lag_llama_04_07/sweeps/8bk4uije


### initialize the estimator

In [5]:
prediction_length = 24
num_samples = 100
device = "cuda:0"

In [6]:
def create_estimator(output_dir):
    
    config = wandb.config
    lr = config.lr
    context_length = config.context_length
    batch_size = config.batch_size

    ckpt = torch.load("lag-llama.ckpt", map_location=torch.device(device))

    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]
    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,
        nonnegative_pred_samples=False,
        aug_prob=0,
        lr=lr,
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        time_feat=estimator_args["time_feat"],
        rope_scaling={
            "type": "linear",
            "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        },
        batch_size=batch_size,
        num_parallel_samples=num_samples,
        trainer_kwargs={"max_epochs": 100},
    #    log_train_loss=True, 
    )

    return estimator

In [7]:
def getCPRS(dataset, predictor, num_samples=100):

    # inference step
    forecast_it, ts_it = make_evaluation_predictions( 
            dataset=dataset,
            predictor=predictor,
            num_samples=num_samples)

    forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))
    tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))
    
    # restore mean and avg for forecasts and tss of dataset (test_ds)
    for i in range(len(tss)):
        tss[i].values[:] = tss[i].values * test_df.std()[i] + test_df.mean()[i]
    for i in range(len(forecasts)):
        samples = forecasts[i].samples
        std_value = test_df.std()[i]
        mean_value = test_df.mean()[i]
        np.multiply(samples, std_value, out=samples)
        np.add(samples, mean_value, out=samples)
    
    # evaluate metrics
    evaluator = Evaluator()
    agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
    
    return agg_metrics['mean_wQuantileLoss']

In [ ]:
def train():
    with wandb.init():
        estimator = create_estimator(output_dir=".")
        predictor = estimator.train(train_ds, cache_data=False, shuffle_buffer_length=1000)
        CPRS = getCPRS(test_ds, predictor, num_samples=100)
        wandb.log({"CPRS": CPRS})
        wandb.finish() # must-have, otherwise w&b doesn't know when a search is finished and thus can't proceed with the next search
# Start the sweep agent 
wandb.agent(sweep_id, train) 

wandb: Agent Starting Run: zinyftfc with config:
wandb: 	batch_size: 2
wandb: 	context_length: 32
wandb: 	lr: 0.0005
wandb: Currently logged in as: junboleng (junboma). Use `wandb login --relogin` to force relogin


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/eragroup/anaconda3/envs/lag_llama/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/home/eragroup/anaconda3/envs/lag_llama/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Para

Training: |                                               | 0/? [00:00<?, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 0.83678 (best 0.83678), saving model to '/home/eragroup/Documents/Documents/Koutas_Master_Thesis_Supervision/Junbo_Leng/lag_llama_model/lag-llama-main/lightning_logs/version_28/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 0.68651 (best 0.68651), saving model to '/home/eragroup/Documents/Documents/Koutas_Master_Thesis_Supervision/Junbo_Leng/lag_llama_model/lag-llama-main/lightning_logs/version_28/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 0.58697 (best 0.58697), saving model to '/home/eragroup/Documents/Documents/Koutas_Master_Thesis_Supervision/Junbo_Leng/lag_llama_model/lag-llama-main/lightning_logs/version_28/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 0.44757 (best 0.44757), saving model to '/home/eragroup/Documents/Documents/Koutas_Master_Thesis_Supervision/Junbo_Leng/lag_llama_model/lag-llama-main/

...

wandb: Agent Starting Run: inyeg6fx with config:
wandb: 	batch_size: 8
wandb: 	context_length: 32
wandb: 	lr: 5e-05


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/eragroup/anaconda3/envs/lag_llama/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | LagLlamaModel      | 2.4 M 
1 | augmentations | ApplyAugmentations | 0     
-----------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.797     Total estimated model params size (MB)


Training: |                                               | 0/? [00:00<?, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 0.46759 (best 0.46759), saving model to '/home/eragroup/Documents/Documents/Koutas_Master_Thesis_Supervision/Junbo_Leng/lag_llama_model/lag-llama-main/lightning_logs/version_46/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 0.35810 (best 0.35810), saving model to '/home/eragroup/Documents/Documents/Koutas_Master_Thesis_Supervision/Junbo_Leng/lag_llama_model/lag-llama-main/lightning_logs/version_46/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' was not in top 1
Epoch 3, global step 200: 'train_loss' was not in top 1
Epoch 4, global step 250: 'train_loss' was not in top 1
Epoch 5, global step 300: 'train_loss' was not in top 1
Epoch 6, global step 350: 'train_loss' reached 0.33221 (best 0.33221), saving model to '/home/eragroup/Documents/Documents/Koutas_Master_Thesis_Supervision/Junbo_Leng/lag_llama_model/lag-llama-main/lightning_logs/version_46/checkpoints/epoch=

### experiment 0: would the format of training datasets have an impact on training results?
- PandasDatasets: CRPS = 0.0311
- ListDatasets: CRPS = 0.0028
  
Conclusion: **stay with ListDatasets**

### experiment 1: would the format of training datasets have an impact on training results?
- #Epochs100: CRPS = 0.0028
- #Epochs500: CRPS = 0.0021

More training epochs
- can improve CPRS score
- the predicitions show more visible fanning-out across the prediciton horizon
- still very much input-independent :(((

### adjusting the learning rate schedule.
### Reducing or removing regularization terms/techniques.
### Tuning the regularization hyperparameters.